In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import os


In [2]:
data = os.path.join(os.getcwd(),'data_final.csv')

In [3]:
df = pd.read_csv(data)
df = df.loc[df['Price movement'].isin(["Up", "Down"])]
df["Headline"].apply(lambda x: x.lower())
df["Headline"].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))

max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(df['Headline'].values)
X = tokenizer.texts_to_sequences(df['Headline'].values)
X = pad_sequences(X)

In [4]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 28, 128)           256000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 28, 128)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
Y = pd.get_dummies(df['Price movement']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(2751, 28) (2751, 2)
(1356, 28) (1356, 2)


In [6]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 20, batch_size=batch_size, verbose = 2)

Epoch 1/20
86/86 - 20s - loss: 0.6884 - accuracy: 0.5413 - 20s/epoch - 229ms/step
Epoch 2/20
86/86 - 6s - loss: 0.6500 - accuracy: 0.6332 - 6s/epoch - 65ms/step
Epoch 3/20
86/86 - 5s - loss: 0.5589 - accuracy: 0.7168 - 5s/epoch - 64ms/step
Epoch 4/20
86/86 - 5s - loss: 0.4673 - accuracy: 0.7801 - 5s/epoch - 64ms/step
Epoch 5/20
86/86 - 6s - loss: 0.3954 - accuracy: 0.8233 - 6s/epoch - 66ms/step
Epoch 6/20
86/86 - 6s - loss: 0.3307 - accuracy: 0.8586 - 6s/epoch - 64ms/step
Epoch 7/20
86/86 - 5s - loss: 0.2963 - accuracy: 0.8768 - 5s/epoch - 63ms/step
Epoch 8/20
86/86 - 5s - loss: 0.2613 - accuracy: 0.8895 - 5s/epoch - 64ms/step
Epoch 9/20
86/86 - 5s - loss: 0.2355 - accuracy: 0.9004 - 5s/epoch - 63ms/step
Epoch 10/20
86/86 - 4s - loss: 0.2123 - accuracy: 0.9138 - 4s/epoch - 41ms/step
Epoch 11/20
86/86 - 4s - loss: 0.2048 - accuracy: 0.9168 - 4s/epoch - 41ms/step
Epoch 12/20
86/86 - 4s - loss: 0.1743 - accuracy: 0.9262 - 4s/epoch - 41ms/step
Epoch 13/20
86/86 - 3s - loss: 0.1630 - accura

In [7]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

43/43 - 1s - loss: 2.3536 - accuracy: 0.5841 - 594ms/epoch - 14ms/step
score: 2.35
acc: 0.58
